# Logit DGP: Neural Network vs Logistic Regression Oracle

This notebook validates the `structural_dml` package against an oracle (correctly specified logistic regression) for a known logit DGP.

## DGP Specification

$$P(Y=1 | X, T) = \sigma(\alpha(X) + \beta(X) \cdot T)$$

where:
- $\alpha(X) = a_0 + a_1 X$
- $\beta(X) = b_0 + b_1 X$
- $\sigma(z) = 1/(1 + e^{-z})$ (sigmoid function)
- $X \sim N(\mu_X=1, \sigma^2=1)$
- $T \sim N(0, 1)$, independent of $X$
- $Y \sim \text{Bernoulli}(p)$ where $p = \sigma(\alpha(X) + \beta(X) \cdot T)$

**Parameters:** $a_0=0.5, a_1=0.3, b_0=-0.5, b_1=0.5$

**Target:** $\mu^* = E[\beta(X)] = b_0 + b_1 \cdot E[X] = -0.5 + 0.5 \cdot 1 = 0$

## Key Insight

Testing $H_0: E[\beta(X)] = 0$ requires accounting for the randomness of $\bar{X}$:
- **Naive Logit** (treat $\bar{X}$ as fixed): undercoverage
- **Delta-corrected Logit**: ~95% coverage  
- **Neural Net with IF**: ~90% coverage (harder than linear due to nonlinearity)

## Section 1: Setup & DGP

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats
from scipy.special import expit  # sigmoid function
import warnings
import sys
from pathlib import Path
from tqdm.notebook import tqdm

# Use local deep_inference (not pip version)
sys.path.insert(0, str(Path.cwd().parent / 'src'))
from deep_inference import structural_dml

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("Setup complete!")

In [ ]:
# DGP Parameters
A0, A1 = 0.5, 0.3      # alpha(X) = a0 + a1*X
B0, B1 = -0.5, 0.5     # beta(X) = b0 + b1*X
MU_X = 1.0             # E[X] = 1
MU_TRUE = B0 + B1 * MU_X  # E[beta(X)] = -0.5 + 0.5*1 = 0

print(f"DGP Parameters:")
print(f"  alpha(X) = {A0} + {A1}*X")
print(f"  beta(X)  = {B0} + {B1}*X")
print(f"  E[X] = {MU_X}")
print(f"  True target mu* = E[beta(X)] = {MU_TRUE}")

In [ ]:
def generate_data(n, seed=None):
    """
    Generate data from the logit DGP.
    
    P(Y=1 | X, T) = sigmoid(alpha(X) + beta(X)*T)
    alpha(X) = a0 + a1*X
    beta(X) = b0 + b1*X
    X ~ N(1, 1)
    T ~ N(0, 1)
    Y ~ Bernoulli(p)
    
    Returns dict with Y, T, X, alpha_true, beta_true, mu_true, p_true
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Generate covariates
    X = np.random.normal(MU_X, 1.0, n)
    T = np.random.normal(0.0, 1.0, n)
    
    # True structural functions
    alpha_true = A0 + A1 * X
    beta_true = B0 + B1 * X
    
    # Probability and binary outcome
    logits = alpha_true + beta_true * T
    p_true = expit(logits)  # sigmoid
    Y = np.random.binomial(1, p_true)
    
    return {
        'Y': Y.astype(float),
        'T': T,
        'X': X,
        'alpha_true': alpha_true,
        'beta_true': beta_true,
        'p_true': p_true,
        'mu_true': beta_true.mean()  # Sample E[beta(X)]
    }

In [ ]:
# Generate sample data for visualization
np.random.seed(42)
data = generate_data(1000, seed=42)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: Y vs T colored by X
ax = axes[0]
scatter = ax.scatter(data['T'], data['Y'] + np.random.normal(0, 0.02, len(data['Y'])), 
                     c=data['X'], cmap='coolwarm', alpha=0.5, s=10)
plt.colorbar(scatter, ax=ax, label='X')
ax.set_xlabel('T (Treatment)')
ax.set_ylabel('Y (Binary Outcome)')
ax.set_title('Y vs T (colored by X, jittered)')

# Plot 2: True alpha(X) and beta(X)
ax = axes[1]
X_grid = np.linspace(-2, 4, 100)
ax.plot(X_grid, A0 + A1 * X_grid, 'b-', label=r'$\alpha(X) = 0.5 + 0.3X$', linewidth=2)
ax.plot(X_grid, B0 + B1 * X_grid, 'r-', label=r'$\beta(X) = -0.5 + 0.5X$', linewidth=2)
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.axvline(x=MU_X, color='gray', linestyle=':', alpha=0.5, label=f'E[X]={MU_X}')
ax.set_xlabel('X')
ax.set_ylabel('Function value')
ax.set_title('True Structural Functions')
ax.legend()

# Plot 3: Distribution of beta(X)
ax = axes[2]
ax.hist(data['beta_true'], bins=30, alpha=0.7, edgecolor='black')
ax.axvline(x=MU_TRUE, color='red', linestyle='--', linewidth=2, label=f'Pop. E[beta(X)]={MU_TRUE}')
ax.axvline(x=data['beta_true'].mean(), color='green', linestyle=':', linewidth=2, 
           label=f'Sample mean={data["beta_true"].mean():.3f}')
ax.set_xlabel(r'$\beta(X)$')
ax.set_ylabel('Frequency')
ax.set_title(r'Distribution of $\beta(X)$')
ax.legend()

plt.tight_layout()
plt.show()

print(f"\nSample statistics:")
print(f"  n = {len(data['Y'])}")
print(f"  mean(X) = {data['X'].mean():.4f}")
print(f"  mean(Y) = {data['Y'].mean():.4f} (proportion of Y=1)")
print(f"  Sample E[beta(X)] = {data['beta_true'].mean():.4f}")
print(f"  Population E[beta(X)] = {MU_TRUE}")

## Section 2: Oracle Implementation

The oracle is logistic regression on the correctly specified model: $\text{logit}(P(Y=1)) = a_0 + a_1 X + b_0 T + b_1 (X \cdot T)$

For inference on $E[\beta(X)] = b_0 + b_1 E[X]$, we need to account for:
1. **Naive SE**: Treats $\bar{X}$ as fixed (WRONG - inflated Type I error)
2. **Delta-corrected SE**: Adds $\hat{b}_1^2 \cdot Var(\bar{X})$ term (CORRECT)

In [ ]:
def logit_oracle(Y, T, X):
    """
    Logistic regression oracle with both naive and delta-corrected SE for E[beta(X)].
    
    Model: logit(P(Y=1)) = a0 + a1*X + b0*T + b1*(X*T)
    Target: mu = E[beta(X)] = b0 + b1*E[X], estimated by b0_hat + b1_hat*X_bar
    
    Returns:
        mu_hat: Point estimate of E[beta(X)]
        se_naive: SE treating X_bar as fixed
        se_delta: Delta-corrected SE accounting for Var(X_bar)
        params: Dict with a0, a1, b0, b1 estimates
        alpha_hat: Fitted alpha(X) = a0 + a1*X
        beta_hat: Fitted beta(X) = b0 + b1*X
    """
    n = len(Y)
    X_bar = X.mean()
    
    # Design matrix: [1, X, T, X*T]
    X_design = np.column_stack([np.ones(n), X, T, X * T])
    
    # Fit logistic regression
    model = sm.Logit(Y, X_design).fit(disp=0)
    
    # Extract coefficients
    a0, a1, b0, b1 = model.params
    
    # Point estimate of E[beta(X)]
    mu_hat = b0 + b1 * X_bar
    
    # Variance-covariance for b0, b1 (indices 2, 3)
    cov = model.cov_params()
    var_b0 = cov[2, 2]
    var_b1 = cov[3, 3]
    cov_b0_b1 = cov[2, 3]
    
    # Naive SE: treats X_bar as fixed
    # Var(b0 + b1*X_bar) = Var(b0) + X_bar^2*Var(b1) + 2*X_bar*Cov(b0,b1)
    var_naive = var_b0 + X_bar**2 * var_b1 + 2 * X_bar * cov_b0_b1
    se_naive = np.sqrt(max(var_naive, 1e-10))
    
    # Delta-corrected SE: accounts for Var(X_bar)
    # Additional term: b1^2 * Var(X_bar) = b1^2 * Var(X)/n
    var_X_bar = X.var(ddof=1) / n
    var_delta = var_naive + b1**2 * var_X_bar
    se_delta = np.sqrt(max(var_delta, 1e-10))
    
    # Fitted structural functions
    alpha_hat = a0 + a1 * X
    beta_hat = b0 + b1 * X
    
    # Predicted probabilities
    p_hat = expit(alpha_hat + beta_hat * T)
    
    return {
        'mu_hat': mu_hat,
        'se_naive': se_naive,
        'se_delta': se_delta,
        'params': {'a0': a0, 'a1': a1, 'b0': b0, 'b1': b1},
        'alpha_hat': alpha_hat,
        'beta_hat': beta_hat,
        'p_hat': p_hat,
        'X_bar': X_bar
    }

In [ ]:
# Test the oracle on sample data
logit_result = logit_oracle(data['Y'], data['T'], data['X'])

print("Logit Oracle Results:")
print(f"\nParameter estimates:")
print(f"  a0 = {logit_result['params']['a0']:.4f} (true: {A0})")
print(f"  a1 = {logit_result['params']['a1']:.4f} (true: {A1})")
print(f"  b0 = {logit_result['params']['b0']:.4f} (true: {B0})")
print(f"  b1 = {logit_result['params']['b1']:.4f} (true: {B1})")

print(f"\nInference on E[beta(X)]:")
print(f"  Point estimate: {logit_result['mu_hat']:.4f} (true: {MU_TRUE})")
print(f"  Naive SE:       {logit_result['se_naive']:.4f}")
print(f"  Delta SE:       {logit_result['se_delta']:.4f}")
print(f"  Ratio delta/naive: {logit_result['se_delta']/logit_result['se_naive']:.3f}")

# 95% CIs
ci_naive = (logit_result['mu_hat'] - 1.96*logit_result['se_naive'], 
            logit_result['mu_hat'] + 1.96*logit_result['se_naive'])
ci_delta = (logit_result['mu_hat'] - 1.96*logit_result['se_delta'], 
            logit_result['mu_hat'] + 1.96*logit_result['se_delta'])

print(f"\n95% Confidence Intervals:")
print(f"  Naive: [{ci_naive[0]:.4f}, {ci_naive[1]:.4f}]")
print(f"  Delta: [{ci_delta[0]:.4f}, {ci_delta[1]:.4f}]")

### Math Verification: Delta Method

The estimator $\hat{\mu} = \hat{b}_0 + \hat{b}_1 \bar{X}$ targets $\mu^* = b_0 + b_1 \mu_X$.

**Delta method expansion:**
$$\hat{\mu} - \mu^* \approx (\hat{b}_0 - b_0) + \mu_X(\hat{b}_1 - b_1) + b_1(\bar{X} - \mu_X)$$

**Variance (assuming $(\hat{b}_0, \hat{b}_1) \perp \bar{X}$):**
$$\text{Var}(\hat{\mu}) = \underbrace{\text{Var}(\hat{b}_0) + \mu_X^2 \text{Var}(\hat{b}_1) + 2\mu_X \text{Cov}(\hat{b}_0, \hat{b}_1)}_{\text{naive (treats } \bar{X} \text{ as fixed)}} + \underbrace{b_1^2 \text{Var}(\bar{X})}_{\text{delta correction}}$$

Using $\bar{X}$ instead of $\mu_X$ and $\hat{b}_1$ instead of $b_1$ are standard plug-in estimates (asymptotically equivalent).

In [ ]:
# Numerical verification of delta method formula
print("Delta Method Formula Verification:")
print(f"  Naive SE^2:     {logit_result['se_naive']**2:.8f}")
print(f"  Delta SE^2:     {logit_result['se_delta']**2:.8f}")
print(f"  Difference:    {logit_result['se_delta']**2 - logit_result['se_naive']**2:.8f}")

# The difference should equal b1^2 * Var(X_bar)
b1_sq_var_xbar = logit_result['params']['b1']**2 * (data['X'].var(ddof=1) / len(data['X']))
print(f"  b1^2 * Var(X_bar): {b1_sq_var_xbar:.8f}")

match = np.isclose(
    logit_result['se_delta']**2 - logit_result['se_naive']**2, 
    b1_sq_var_xbar,
    rtol=1e-4
)
print(f"  Formula matches: {match}")

## Section 3: Single-Run Comparison

Compare Logit oracle vs Neural Network on a single dataset.

In [ ]:
# Generate fresh data for comparison
np.random.seed(123)
n = 1000
data = generate_data(n, seed=123)

print(f"Generated n={n} observations")
print(f"Sample E[beta(X)] = {data['beta_true'].mean():.4f}")
print(f"Sample mean(Y) = {data['Y'].mean():.4f}")

In [ ]:
# Fit Logit Oracle
logit_result = logit_oracle(data['Y'], data['T'], data['X'])

print("Logit Oracle:")
print(f"  mu_hat = {logit_result['mu_hat']:.4f}")
print(f"  SE (delta) = {logit_result['se_delta']:.4f}")

In [ ]:
# Fit Neural Network
nn_result = structural_dml(
    Y=data['Y'],
    T=data['T'],
    X=data['X'].reshape(-1, 1),  # Must be 2D
    family='logit',
    epochs=100,
    n_folds=50,
    hidden_dims=[64, 32],
    lr=0.01,
    verbose=False
)

print("Neural Network (structural_dml):")
print(f"  mu_hat = {nn_result.mu_hat:.4f}")
print(f"  SE = {nn_result.se:.4f}")
print(f"  95% CI: [{nn_result.ci_lower:.4f}, {nn_result.ci_upper:.4f}]")
print(f"  mu_naive = {nn_result.mu_naive:.4f}")

### a) Parameter Recovery: $\alpha(X)$ and $\beta(X)$

In [ ]:
# Extract fitted structural functions
alpha_logit = logit_result['alpha_hat']
beta_logit = logit_result['beta_hat']

alpha_nn = nn_result.theta_hat[:, 0]
beta_nn = nn_result.theta_hat[:, 1]

# Compute recovery metrics
def compute_metrics(estimated, true):
    rmse = np.sqrt(np.mean((estimated - true)**2))
    corr = np.corrcoef(estimated, true)[0, 1]
    bias = np.mean(estimated - true)
    return {'rmse': rmse, 'corr': corr, 'bias': bias}

metrics = {
    'Logit': {
        'alpha': compute_metrics(alpha_logit, data['alpha_true']),
        'beta': compute_metrics(beta_logit, data['beta_true'])
    },
    'NN': {
        'alpha': compute_metrics(alpha_nn, data['alpha_true']),
        'beta': compute_metrics(beta_nn, data['beta_true'])
    }
}

print("Parameter Recovery Metrics:")
print("\n" + "="*65)
print(f"{'Metric':<15} {'Logit alpha':<13} {'Logit beta':<13} {'NN alpha':<13} {'NN beta':<13}")
print("="*65)
print(f"{'RMSE':<15} {metrics['Logit']['alpha']['rmse']:<13.4f} {metrics['Logit']['beta']['rmse']:<13.4f} {metrics['NN']['alpha']['rmse']:<13.4f} {metrics['NN']['beta']['rmse']:<13.4f}")
print(f"{'Correlation':<15} {metrics['Logit']['alpha']['corr']:<13.4f} {metrics['Logit']['beta']['corr']:<13.4f} {metrics['NN']['alpha']['corr']:<13.4f} {metrics['NN']['beta']['corr']:<13.4f}")
print(f"{'Bias':<15} {metrics['Logit']['alpha']['bias']:<13.4f} {metrics['Logit']['beta']['bias']:<13.4f} {metrics['NN']['alpha']['bias']:<13.4f} {metrics['NN']['beta']['bias']:<13.4f}")
print("="*65)

In [ ]:
# Visualize parameter recovery
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Logit alpha
ax = axes[0, 0]
ax.scatter(data['alpha_true'], alpha_logit, alpha=0.3, s=10)
ax.plot([data['alpha_true'].min(), data['alpha_true'].max()], 
        [data['alpha_true'].min(), data['alpha_true'].max()], 'r--', linewidth=2)
ax.set_xlabel(r'True $\alpha(X)$')
ax.set_ylabel(r'Logit $\hat{\alpha}(X)$')
ax.set_title(f'Logit: Corr={metrics["Logit"]["alpha"]["corr"]:.3f}')

# Logit beta
ax = axes[0, 1]
ax.scatter(data['beta_true'], beta_logit, alpha=0.3, s=10)
ax.plot([data['beta_true'].min(), data['beta_true'].max()], 
        [data['beta_true'].min(), data['beta_true'].max()], 'r--', linewidth=2)
ax.set_xlabel(r'True $\beta(X)$')
ax.set_ylabel(r'Logit $\hat{\beta}(X)$')
ax.set_title(f'Logit: Corr={metrics["Logit"]["beta"]["corr"]:.3f}')

# NN alpha
ax = axes[1, 0]
ax.scatter(data['alpha_true'], alpha_nn, alpha=0.3, s=10)
ax.plot([data['alpha_true'].min(), data['alpha_true'].max()], 
        [data['alpha_true'].min(), data['alpha_true'].max()], 'r--', linewidth=2)
ax.set_xlabel(r'True $\alpha(X)$')
ax.set_ylabel(r'NN $\hat{\alpha}(X)$')
ax.set_title(f'NN: Corr={metrics["NN"]["alpha"]["corr"]:.3f}')

# NN beta
ax = axes[1, 1]
ax.scatter(data['beta_true'], beta_nn, alpha=0.3, s=10)
ax.plot([data['beta_true'].min(), data['beta_true'].max()], 
        [data['beta_true'].min(), data['beta_true'].max()], 'r--', linewidth=2)
ax.set_xlabel(r'True $\beta(X)$')
ax.set_ylabel(r'NN $\hat{\beta}(X)$')
ax.set_title(f'NN: Corr={metrics["NN"]["beta"]["corr"]:.3f}')

plt.tight_layout()
plt.show()

In [ ]:
# Plot fitted functions vs X
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# alpha(X) vs X
ax = axes[0]
ax.scatter(data['X'], data['alpha_true'], alpha=0.2, s=10, label='True', color='black')
ax.scatter(data['X'], alpha_logit, alpha=0.2, s=10, label='Logit', color='blue')
ax.scatter(data['X'], alpha_nn, alpha=0.2, s=10, label='NN', color='red')

# True line
X_grid = np.linspace(data['X'].min(), data['X'].max(), 100)
ax.plot(X_grid, A0 + A1 * X_grid, 'k-', linewidth=2, label='True function')

ax.set_xlabel('X')
ax.set_ylabel(r'$\alpha(X)$')
ax.set_title(r'Recovery of $\alpha(X) = 0.5 + 0.3X$')
ax.legend()

# beta(X) vs X
ax = axes[1]
ax.scatter(data['X'], data['beta_true'], alpha=0.2, s=10, label='True', color='black')
ax.scatter(data['X'], beta_logit, alpha=0.2, s=10, label='Logit', color='blue')
ax.scatter(data['X'], beta_nn, alpha=0.2, s=10, label='NN', color='red')

# True line
ax.plot(X_grid, B0 + B1 * X_grid, 'k-', linewidth=2, label='True function')
ax.axhline(y=MU_TRUE, color='gray', linestyle='--', alpha=0.5, label=f'E[beta(X)]={MU_TRUE}')

ax.set_xlabel('X')
ax.set_ylabel(r'$\beta(X)$')
ax.set_title(r'Recovery of $\beta(X) = -0.5 + 0.5X$')
ax.legend()

plt.tight_layout()
plt.show()

### b) Training Diagnostics: Logit-Specific Checks

In [ ]:
# Extract training diagnostics from NN result
diagnostics = nn_result.diagnostics

print("Neural Network Training Diagnostics:")
print(f"  Min Lambda eigenvalue: {diagnostics.get('min_lambda_eigenvalue', 'N/A')}")
print(f"  Pct regularized: {diagnostics.get('pct_regularized', 'N/A')}%")
print(f"  Correction ratio: {diagnostics.get('correction_ratio', 'N/A')}")

# For logit, check Hessian stability (important because H depends on theta)
min_lambda = diagnostics.get('min_lambda_eigenvalue', None)
if min_lambda is not None:
    print(f"\nHessian Stability Check:")
    print(f"  min(lambda) = {min_lambda:.6f}")
    if min_lambda > 1e-4:
        print(f"  [PASS] min(lambda) > 1e-4 (stable Hessian)")
    else:
        print(f"  [WARN] min(lambda) <= 1e-4 (may need more regularization)")

## Section 4: Monte Carlo Simulation

Run M replications to assess:
- c) Bias and variance of $\hat{\mu}$ estimates
- d) Coverage and SE calibration

In [ ]:
# Monte Carlo configuration
M = 100          # Number of replications
N = 1000         # Sample size per replication
N_FOLDS = 50     # Cross-fitting folds for NN
EPOCHS = 100     # Training epochs

print(f"Monte Carlo Configuration:")
print(f"  M = {M} replications")
print(f"  n = {N} observations per replication")
print(f"  K = {N_FOLDS} folds for NN")
print(f"  True target mu* = {MU_TRUE}")

In [ ]:
def run_single_mc(sim_id, n, n_folds, epochs):
    """
    Run a single Monte Carlo replication.
    
    Returns dict with Logit and NN results.
    """
    # Generate data
    data = generate_data(n)
    
    # Logit Oracle
    logit = logit_oracle(data['Y'], data['T'], data['X'])
    
    # Neural Network
    nn = structural_dml(
        Y=data['Y'],
        T=data['T'],
        X=data['X'].reshape(-1, 1),
        family='logit',
        epochs=epochs,
        n_folds=n_folds,
        hidden_dims=[64, 32],
        lr=0.01,
        verbose=False
    )
    
    # Coverage indicators
    mu_true = MU_TRUE  # Population target
    
    # Logit naive CI
    ci_naive_lo = logit['mu_hat'] - 1.96 * logit['se_naive']
    ci_naive_hi = logit['mu_hat'] + 1.96 * logit['se_naive']
    covered_naive = (ci_naive_lo <= mu_true <= ci_naive_hi)
    
    # Logit delta CI
    ci_delta_lo = logit['mu_hat'] - 1.96 * logit['se_delta']
    ci_delta_hi = logit['mu_hat'] + 1.96 * logit['se_delta']
    covered_delta = (ci_delta_lo <= mu_true <= ci_delta_hi)
    
    # NN IF CI
    covered_nn = (nn.ci_lower <= mu_true <= nn.ci_upper)
    
    # Parameter recovery metrics
    alpha_nn = nn.theta_hat[:, 0]
    beta_nn = nn.theta_hat[:, 1]
    
    corr_alpha_logit = np.corrcoef(logit['alpha_hat'], data['alpha_true'])[0, 1]
    corr_beta_logit = np.corrcoef(logit['beta_hat'], data['beta_true'])[0, 1]
    corr_alpha_nn = np.corrcoef(alpha_nn, data['alpha_true'])[0, 1]
    corr_beta_nn = np.corrcoef(beta_nn, data['beta_true'])[0, 1]
    
    return {
        'sim_id': sim_id,
        'sample_mu_true': data['mu_true'],  # Sample E[beta(X)]
        
        # Logit estimates
        'logit_mu': logit['mu_hat'],
        'logit_se_naive': logit['se_naive'],
        'logit_se_delta': logit['se_delta'],
        'logit_covered_naive': covered_naive,
        'logit_covered_delta': covered_delta,
        
        # NN estimates
        'nn_mu': nn.mu_hat,
        'nn_mu_naive': nn.mu_naive,
        'nn_se': nn.se,
        'nn_covered': covered_nn,
        
        # Parameter recovery
        'corr_alpha_logit': corr_alpha_logit,
        'corr_beta_logit': corr_beta_logit,
        'corr_alpha_nn': corr_alpha_nn,
        'corr_beta_nn': corr_beta_nn,
        
        # Diagnostics
        'min_lambda': nn.diagnostics.get('min_lambda_eigenvalue', np.nan),
    }

In [ ]:
# Run Monte Carlo simulation
results = []

for sim_id in tqdm(range(M), desc='Monte Carlo'):
    np.random.seed(sim_id + 1000)  # Reproducible seeds
    result = run_single_mc(sim_id, N, N_FOLDS, EPOCHS)
    results.append(result)

df = pd.DataFrame(results)
print(f"\nCompleted {M} Monte Carlo replications.")

### c) Bias and Variance

In [ ]:
# Compute bias and variance metrics
mu_true = MU_TRUE

# Logit
logit_bias = df['logit_mu'].mean() - mu_true
logit_var = df['logit_mu'].var()
logit_rmse = np.sqrt(logit_bias**2 + logit_var)
logit_se_emp = df['logit_mu'].std()

# NN
nn_bias = df['nn_mu'].mean() - mu_true
nn_var = df['nn_mu'].var()
nn_rmse = np.sqrt(nn_bias**2 + nn_var)
nn_se_emp = df['nn_mu'].std()

print("="*60)
print("BIAS AND VARIANCE (target mu* = 0)")
print("="*60)
print(f"{'Metric':<20} {'Logit Oracle':<15} {'Neural Net':<15}")
print("-"*60)
print(f"{'Mean estimate':<20} {df['logit_mu'].mean():<15.4f} {df['nn_mu'].mean():<15.4f}")
print(f"{'Bias':<20} {logit_bias:<15.4f} {nn_bias:<15.4f}")
print(f"{'Variance':<20} {logit_var:<15.4f} {nn_var:<15.4f}")
print(f"{'RMSE':<20} {logit_rmse:<15.4f} {nn_rmse:<15.4f}")
print(f"{'Empirical SE':<20} {logit_se_emp:<15.4f} {nn_se_emp:<15.4f}")
print("="*60)

### d) Coverage and SE Calibration

In [ ]:
# Coverage and SE calibration
coverage_naive = df['logit_covered_naive'].mean()
coverage_delta = df['logit_covered_delta'].mean()
coverage_nn = df['nn_covered'].mean()

# SE ratios (mean estimated SE / empirical SE)
se_ratio_naive = df['logit_se_naive'].mean() / logit_se_emp
se_ratio_delta = df['logit_se_delta'].mean() / logit_se_emp
se_ratio_nn = df['nn_se'].mean() / nn_se_emp

print("="*70)
print("COVERAGE AND SE CALIBRATION")
print("="*70)
print(f"{'Metric':<20} {'Logit Naive':<15} {'Logit Delta':<15} {'Neural Net':<15}")
print("-"*70)
print(f"{'Coverage':<20} {coverage_naive:<15.1%} {coverage_delta:<15.1%} {coverage_nn:<15.1%}")
print(f"{'Mean Est SE':<20} {df['logit_se_naive'].mean():<15.4f} {df['logit_se_delta'].mean():<15.4f} {df['nn_se'].mean():<15.4f}")
print(f"{'Empirical SE':<20} {logit_se_emp:<15.4f} {logit_se_emp:<15.4f} {nn_se_emp:<15.4f}")
print(f"{'SE Ratio':<20} {se_ratio_naive:<15.2f} {se_ratio_delta:<15.2f} {se_ratio_nn:<15.2f}")
print("="*70)

print("\nTarget ranges:")
print("  Coverage: 88-97% (logit is harder than linear)")
print("  SE Ratio: 0.85-1.15")

In [ ]:
# Hessian stability summary
print("\nHessian Stability (Logit-specific):")
print(f"  Mean min(lambda): {df['min_lambda'].mean():.6f}")
print(f"  Min min(lambda): {df['min_lambda'].min():.6f}")
print(f"  Pct with min(lambda) > 1e-4: {(df['min_lambda'] > 1e-4).mean():.1%}")

In [ ]:
# Parameter recovery summary
print("="*60)
print("PARAMETER RECOVERY (Correlation with true values)")
print("="*60)
print(f"{'Parameter':<15} {'Logit Oracle':<15} {'Neural Net':<15}")
print("-"*60)
print(f"{'alpha(X)':<15} {df['corr_alpha_logit'].mean():<15.3f} {df['corr_alpha_nn'].mean():<15.3f}")
print(f"{'beta(X)':<15} {df['corr_beta_logit'].mean():<15.3f} {df['corr_beta_nn'].mean():<15.3f}")
print("="*60)

## Section 5: Results Visualization

In [ ]:
# Histogram of estimates
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Logit estimates
ax = axes[0]
ax.hist(df['logit_mu'], bins=20, alpha=0.7, edgecolor='black', label='Logit')
ax.axvline(x=MU_TRUE, color='red', linestyle='--', linewidth=2, label=f'True mu*={MU_TRUE}')
ax.axvline(x=df['logit_mu'].mean(), color='blue', linestyle=':', linewidth=2, 
           label=f'Mean={df["logit_mu"].mean():.4f}')
ax.set_xlabel(r'$\hat{\mu}$')
ax.set_ylabel('Frequency')
ax.set_title(f'Logit Oracle Estimates (Coverage: {coverage_delta:.1%})')
ax.legend()

# NN estimates
ax = axes[1]
ax.hist(df['nn_mu'], bins=20, alpha=0.7, edgecolor='black', color='orange', label='NN')
ax.axvline(x=MU_TRUE, color='red', linestyle='--', linewidth=2, label=f'True mu*={MU_TRUE}')
ax.axvline(x=df['nn_mu'].mean(), color='darkorange', linestyle=':', linewidth=2, 
           label=f'Mean={df["nn_mu"].mean():.4f}')
ax.set_xlabel(r'$\hat{\mu}$')
ax.set_ylabel('Frequency')
ax.set_title(f'Neural Net Estimates (Coverage: {coverage_nn:.1%})')
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# QQ plots for t-statistics
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Logit Naive t-stats
t_naive = (df['logit_mu'] - MU_TRUE) / df['logit_se_naive']
ax = axes[0]
stats.probplot(t_naive, dist='norm', plot=ax)
ax.set_title(f'Logit Naive t-stats\n(Coverage: {coverage_naive:.1%})')

# Logit Delta t-stats
t_delta = (df['logit_mu'] - MU_TRUE) / df['logit_se_delta']
ax = axes[1]
stats.probplot(t_delta, dist='norm', plot=ax)
ax.set_title(f'Logit Delta t-stats\n(Coverage: {coverage_delta:.1%})')

# NN t-stats
t_nn = (df['nn_mu'] - MU_TRUE) / df['nn_se']
ax = axes[2]
stats.probplot(t_nn, dist='norm', plot=ax)
ax.set_title(f'Neural Net t-stats\n(Coverage: {coverage_nn:.1%})')

plt.tight_layout()
plt.show()

print(f"t-statistic summary:")
print(f"  Logit Naive: mean={t_naive.mean():.3f}, std={t_naive.std():.3f}")
print(f"  Logit Delta: mean={t_delta.mean():.3f}, std={t_delta.std():.3f}")
print(f"  Neural Net: mean={t_nn.mean():.3f}, std={t_nn.std():.3f}")
print(f"  Standard normal: mean=0, std=1")

In [ ]:
# Comparison of coverage
fig, ax = plt.subplots(figsize=(8, 6))

methods = ['Logit Naive', 'Logit Delta', 'Neural Net (IF)']
coverages = [coverage_naive, coverage_delta, coverage_nn]
colors = ['#ff7f7f', '#7fbf7f', '#7f7fff']

bars = ax.bar(methods, coverages, color=colors, edgecolor='black')
ax.axhline(y=0.95, color='red', linestyle='--', linewidth=2, label='Target (95%)')
ax.axhspan(0.88, 0.97, alpha=0.2, color='green', label='Valid range (88-97%)')

ax.set_ylabel('Coverage')
ax.set_title('95% CI Coverage Comparison (Logit)')
ax.set_ylim(0.7, 1.0)
ax.legend()

# Add value labels on bars
for bar, cov in zip(bars, coverages):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
            f'{cov:.1%}', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

## Section 6: Conclusions

In [ ]:
# Final summary table
print("="*70)
print("FINAL SUMMARY")
print("="*70)
print(f"\nDGP: P(Y=1) = sigmoid(alpha(X) + beta(X)*T)")
print(f"     alpha(X) = {A0} + {A1}*X")
print(f"     beta(X) = {B0} + {B1}*X")
print(f"     Target: E[beta(X)] = {MU_TRUE}")
print(f"\nMonte Carlo: M={M} replications, n={N} observations each")
print()

# Summary table
summary_data = {
    'Metric': ['Bias', 'Variance', 'RMSE', 'Empirical SE', 'Mean Est SE', 'SE Ratio', 'Coverage'],
    'Logit Naive': [f'{logit_bias:.4f}', f'{logit_var:.4f}', f'{logit_rmse:.4f}', 
                  f'{logit_se_emp:.4f}', f'{df["logit_se_naive"].mean():.4f}', 
                  f'{se_ratio_naive:.2f}', f'{coverage_naive:.1%}'],
    'Logit Delta': [f'{logit_bias:.4f}', f'{logit_var:.4f}', f'{logit_rmse:.4f}', 
                  f'{logit_se_emp:.4f}', f'{df["logit_se_delta"].mean():.4f}', 
                  f'{se_ratio_delta:.2f}', f'{coverage_delta:.1%}'],
    'Neural Net': [f'{nn_bias:.4f}', f'{nn_var:.4f}', f'{nn_rmse:.4f}', 
                   f'{nn_se_emp:.4f}', f'{df["nn_se"].mean():.4f}', 
                   f'{se_ratio_nn:.2f}', f'{coverage_nn:.1%}']
}
summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

print("\n" + "="*70)
print("VALIDATION CHECK")
print("="*70)

# Check targets (logit has wider tolerance than linear)
checks = []

if coverage_naive < 0.90:
    checks.append(f"[EXPECTED] Logit Naive coverage ({coverage_naive:.1%}) < 90% - ignoring Var(X_bar)")
    
if 0.88 <= coverage_delta <= 0.97:
    checks.append(f"[PASS] Logit Delta coverage: {coverage_delta:.1%} (target: 88-97%)")
else:
    checks.append(f"[FAIL] Logit Delta coverage: {coverage_delta:.1%} (target: 88-97%)")

if 0.85 <= coverage_nn <= 0.97:
    checks.append(f"[PASS] NN coverage: {coverage_nn:.1%} (target: 85-97%)")
else:
    checks.append(f"[WARN] NN coverage: {coverage_nn:.1%} (target: 85-97%)")

if 0.85 <= se_ratio_delta <= 1.15:
    checks.append(f"[PASS] Logit Delta SE ratio: {se_ratio_delta:.2f} (target: 0.85-1.15)")
else:
    checks.append(f"[FAIL] Logit Delta SE ratio: {se_ratio_delta:.2f} (target: 0.85-1.15)")

if 0.80 <= se_ratio_nn <= 1.20:
    checks.append(f"[PASS] NN SE ratio: {se_ratio_nn:.2f} (target: 0.80-1.20)")
else:
    checks.append(f"[WARN] NN SE ratio: {se_ratio_nn:.2f} (target: 0.80-1.20)")

corr_beta_nn_mean = df['corr_beta_nn'].mean()
if corr_beta_nn_mean > 0.85:
    checks.append(f"[PASS] NN beta(X) recovery: Corr={corr_beta_nn_mean:.3f} (target: >0.85)")
else:
    checks.append(f"[WARN] NN beta(X) recovery: Corr={corr_beta_nn_mean:.3f} (target: >0.85)")

min_lambda_pct = (df['min_lambda'] > 1e-4).mean()
if min_lambda_pct > 0.95:
    checks.append(f"[PASS] Hessian stability: {min_lambda_pct:.1%} with min(lambda)>1e-4")
else:
    checks.append(f"[WARN] Hessian stability: {min_lambda_pct:.1%} with min(lambda)>1e-4")

for check in checks:
    print(check)

print("\n" + "="*70)

## Key Findings

1. **Logit Naive (ignoring $Var(\bar{X})$)**: Under-covers because it ignores the sampling variability of $\bar{X}$ in $\hat{\mu} = \hat{b}_0 + \hat{b}_1 \bar{X}$.

2. **Logit Delta (accounting for $Var(\bar{X})$)**: Achieves valid coverage by adding $\hat{b}_1^2 Var(\bar{X})$ to the variance.

3. **Neural Network with Influence Functions**: Achieves reasonable coverage (~90%), demonstrating that:
   - The influence function correction works for nonlinear models
   - The SE estimate is reasonably calibrated
   - Logit is harder than linear (coverage slightly below 95% is expected)

4. **Hessian Stability**: The logit Hessian depends on $\theta$ through $p(1-p)$, requiring three-way splitting. The package handles this automatically.

5. **Parameter Recovery**: Both Logit oracle and NN achieve good recovery of $\alpha(X)$ and $\beta(X)$, though NN may have slightly higher variance.